# European Develompent Indicators - Iceland

## Research Questions

1. How has Iceland's GDP growth correlated with inflation rates over the past 20 years?

2. What is the relationship between educational attainment and unemployment in Iceland?

3. What is the gender wage gap trend in Iceland, and how does it compare with female employment rates?

## Datasets

* [namq_10_gdp](https://doi.org/10.2908/NAMQ_10_GDP) - Eurostat: Quarterly GDP

* [nama_10_co3_p3](https://doi.org/10.2908/NAMA_10_CO3_P3) - Eurostat: Annual household disposable income

* [tec00001](https://doi.org/10.2908/TEC00001) - Eurostat: Annual GDP at market prices

* [prc_hicp_midx](https://doi.org/10.2908/PRC_HICP_MIDX) - Eurostat: Monthly Harmonized CPI

* [prc_hicp_manr](https://doi.org/10.2908/PRC_HICP_MANR) - Eurostat: Monthly CPI inflation rate

* [prc_hicp_inw](https://doi.org/10.2908/PRC_HICP_INW) - Eurostat: Annual CPI inflation rate by income group

* [lfsa_urgaed](https://doi.org/10.2908/LFSA_URGAED) - Eurostat: Annual unemployment rate by education level

* [lfsa_egaed](https://doi.org/10.2908/LFSA_EGAED) - Eurostat: Annual employment rate by education level

* [lfsa_egised](https://doi.org/10.2908/LFSA_EGISED) - Eurostat: Annual employed persons by occupation and educational attainment level

* [lfsa_egan](https://doi.org/10.2908/LFSA_EGAN) - Eurostat: Annual employment

* [lfsi_emp_q](https://doi.org/10.2908/LFSI_EMP_Q) - Eurostat: Quarterly employment and activity by sex and age

* [lfsi_long_q](https://doi.org/10.2908/LFSI_LONG_Q) - Eurostat: Quarterly labour market transitions

* [lfsq_ergan](https://doi.org/10.2908/LFSQ_ERGAN) - Eurostat: Quarterly employment rates by citizenship

* [une_rt_m](https://doi.org/10.2908/UNE_RT_M) - Eurostat: Monthly total unemployment rate

* [edat_lfse_03](https://doi.org/10.2908/EDAT_LFSE_03) - Eurostat: Annual population by educational attainment level

* [earn_nt_net](https://doi.org/10.2908/EARN_NT_NET) - Eurostat: Annual net earnings

* [earn_gr_gpgr2](https://doi.org/10.2908/EARN_GR_GPGR2) - Eurostat: Annual gender pay gap in unadjusted form


## Imports

In [1]:
import os
import eurostat
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from src.frame_class import Frame, Frequency, Category

## Parameters

In [2]:
COUNTRY = {
    "name": "Iceland",
    "code": "IS"
}
DATAFRAMES = [
    Frame(
        name="Quarterly GDP",
        file_name="quarterly_gdp.csv",
        description="Gross domestic product (GDP) and main components (output, expenditure and income)",
        eurostat_code="namq_10_gdp",
        frequency=Frequency.QUARTERLY,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual Household Disposable Income",
        file_name="annual_household_disposable_income.csv",
        description="Household final consumption expenditure by purpose (COICOP 1999)",
        eurostat_code="nama_10_co3_p3",
        frequency=Frequency.ANNUAL,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Annual GDP at Market Prices",
        file_name="annual_gdp_at_market_prices.csv",
        description="Gross domestic product at market prices",
        eurostat_code="tec00001",
        frequency=Frequency.ANNUAL,
        categories=[Category.MACROECONOMICS]
    ),
    Frame(
        name="Monthly Harmonized CPI",
        file_name="monthly_harmonized_cpi.csv",
        description="HICP - monthly data (index)",
        eurostat_code="prc_hicp_midx",
        frequency=Frequency.MONTHLY,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Monthly CPI Inflation Rate",
        file_name="monthly_cpi_inflation_rate.csv",
        description="HICP - monthly data (annual rate of change)",
        eurostat_code="prc_hicp_manr",
        frequency=Frequency.MONTHLY,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Annual CPI Inflation Rate by Income Group",
        file_name="annual_cpi_inflation_rate_by_income_group.csv",
        description="HICP - item weights",
        eurostat_code="prc_hicp_inw",
        frequency=Frequency.ANNUAL,
        categories=[Category.PRICES_AND_INFLATION]
    ),
    Frame(
        name="Annual Unemployment Rate by Education Level",
        file_name="annual_unemployment_rate_by_education_level.csv",
        description="Unemployment rates by educational attainment level",
        eurostat_code="lfsa_urgaed",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employment Rate by Education Level",
        file_name="annual_employment_rate_by_education_level.csv",
        description="Employed persons by educational attainment level",
        eurostat_code="lfsa_egaed",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employed Persons by Occupation and Educational Attainment Level",
        file_name="annual_employed_persons_by_occupation_and_educational_attainment_level.csv",
        description="Employed persons by occupation and educational attainment level",
        eurostat_code="lfsa_egised",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET, Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual Employment",
        file_name="annual_employment.csv",
        description="Employed persons by citizenship",
        eurostat_code="lfsa_egan",
        frequency=Frequency.ANNUAL,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment and activity by sex and age",
        file_name="quarterly_employment_and_activity_by_sex_and_age.csv",
        description="Employment and activity by sex and age - quarterly data",
        eurostat_code="lfsi_emp_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly labour market transitions",
        file_name="quarterly_labour_market_transitions.csv",
        description="Labour market transitions - quarterly data",
        eurostat_code="lfsi_long_q",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Quarterly employment rates by citizenship",
        file_name="quarterly_employment_rates_by_citizenship.csv",
        description="Employment rates by citizenship - quarterly data",
        eurostat_code="lfsq_ergan",
        frequency=Frequency.QUARTERLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Monthly total unemployment rate",
        file_name="monthly_total_unemployment_rate.csv",
        description="Unemployment by sex and age - monthly data",
        eurostat_code="une_rt_m",
        frequency=Frequency.MONTHLY,
        categories=[Category.LABOR_MARKET]
    ),
    Frame(
        name="Annual population by educational attainment level",
        file_name="annual_population_by_educational_attainment_level.csv",
        description="Population in private households by educational attainment level - main indicators",
        eurostat_code="edat_lfse_03",
        frequency=Frequency.ANNUAL,
        categories=[Category.EDUCATION_AND_HUMAN_CAPITAL]
    ),
    Frame(
        name="Annual net earnings",
        file_name="annual_net_earnings.csv",
        description="Annual net earnings",
        eurostat_code="earn_nt_net",
        frequency=Frequency.ANNUAL,
        categories=[Category.EARNINGS_AND_INEQUALITY]
    ),
    Frame(
        name="Annual gender pay gap in unadjusted form",
        file_name="annual_gender_pay_gap_in_unadjusted_form.csv",
        description="Gender pay gap in unadjusted form by NACE Rev. 2 activity - structure of earnings survey methodology",
        eurostat_code="earn_gr_gpgr2",
        frequency=Frequency.ANNUAL,
        categories=[Category.EARNINGS_AND_INEQUALITY]
    )
]

DATA_PATH = "data/"

print(f"Frame count: {len(DATAFRAMES)}")

Frame count: 17


In [3]:
for frame in DATAFRAMES:
    df: pd.DataFrame | None = None

    if os.path.exists(os.path.join("original_data", frame.file_name)):
        df = pd.read_csv(os.path.join("original_data", frame.file_name))
        print(f"Loaded {frame.name} dataset from local CSV.")

    else:
        try:
            df = eurostat.get_data_df(frame.eurostat_code, flags=False)
            os.makedirs("original_data", exist_ok=True)
            df.to_csv(os.path.join("original_data", frame.file_name), index=False)

            print(f"Downloaded and saved {frame.name} dataset.")

        except Exception as e:
            print(f"Failed to download {frame.name}: {e}")

    df.rename(columns={"geo\\TIME_PERIOD": "geo"}, inplace=True)
    df = df[df["geo"] == COUNTRY["code"]]
    df = df.dropna(axis=1, how='all')

    if "currency" in df.columns:
        df.rename(columns={"currency": "unit"}, inplace=True)

    if "freq" in df.columns:
        df = df.drop(columns=["freq"])

    if "unit" not in df.columns:
        df.insert(0, "unit", "rate")

    frame.dataframe = df

    os.makedirs(DATA_PATH, exist_ok=True)
    frame.dataframe.to_csv(os.path.join(DATA_PATH, frame.file_name), index=False)


Loaded Quarterly GDP dataset from local CSV.
Loaded Annual Household Disposable Income dataset from local CSV.
Loaded Annual GDP at Market Prices dataset from local CSV.
Loaded Monthly Harmonized CPI dataset from local CSV.
Loaded Monthly CPI Inflation Rate dataset from local CSV.
Loaded Annual CPI Inflation Rate by Income Group dataset from local CSV.
Loaded Annual Unemployment Rate by Education Level dataset from local CSV.
Loaded Annual Employment Rate by Education Level dataset from local CSV.
Loaded Annual Employed Persons by Occupation and Educational Attainment Level dataset from local CSV.
Loaded Annual Employment dataset from local CSV.
Loaded Quarterly employment and activity by sex and age dataset from local CSV.
Loaded Quarterly labour market transitions dataset from local CSV.
Loaded Quarterly employment rates by citizenship dataset from local CSV.
Loaded Monthly total unemployment rate dataset from local CSV.
Loaded Annual population by educational attainment level datase

In [4]:
for frame in DATAFRAMES:
    column_names = frame.dataframe.columns.tolist()
    time_period_index = column_names.index("geo")
    data_columns = column_names[time_period_index + 1 :]

    if frame.frequency == Frequency.MONTHLY:
        frame.dataframe.rename(columns={col: f"{col[:4]}-{col[5:7]}-01T00:00:00" for col in data_columns}, inplace=True)
    elif frame.frequency == Frequency.QUARTERLY:
        frame.dataframe.rename(columns={col: f"{col[:4]}-{(int(col[6]) - 1) * 3 + 1:02d}-01T00:00:00" for col in data_columns}, inplace=True)
    elif frame.frequency == Frequency.ANNUAL:
        frame.dataframe.rename(columns={col: f"{col}-01-01T00:00:00" for col in data_columns}, inplace=True)

    display(frame.dataframe.head(10))

,unit,s_adj,na_item,geo,1995-01-01T00:00:00,1995-04-01T00:00:00,1995-07-01T00:00:00,1995-10-01T00:00:00,1996-01-01T00:00:00,1996-04-01T00:00:00,...,2023-01-01T00:00:00,2023-04-01T00:00:00,2023-07-01T00:00:00,2023-10-01T00:00:00,2024-01-01T00:00:00,2024-04-01T00:00:00,2024-07-01T00:00:00,2024-10-01T00:00:00,2025-01-01T00:00:00,2025-04-01T00:00:00
242,CLV05_MEUR,NSA,B1GQ,IS,2030.6,2194.0,2303.3,2313.8,2200.1,2288.7,...,5146.2,5278.0,5470.4,5355.3,5033.4,5288.4,5432.6,5289.3,5168.4,5187.4
369,CLV05_MEUR,NSA,P3,IS,1656.3,1799.1,1829.6,1913.9,1723.3,1874.8,...,3803.0,3906.5,3679.8,3937.2,3814.3,3931.8,3759.9,4011.0,3914.6,4016.6
409,CLV05_MEUR,NSA,P31_S13,IS,384.5,393.5,407.1,416.2,389.3,398.4,...,686.8,701.7,692.8,715.0,693.2,716.6,707.9,730.8,709.2,719.0
447,CLV05_MEUR,NSA,P31_S14,IS,1004.4,1123.8,1134.6,1200.3,1060.7,1188.3,...,2602.4,2669.2,2441.5,2676.8,2600.4,2663.4,2495.3,2722.2,2674.0,2750.0
488,CLV05_MEUR,NSA,P31_S14_S15,IS,1051.2,1171.8,1183.0,1249.5,1107.0,1235.8,...,2683.7,2764.5,2539.9,2770.4,2682.8,2761.5,2595.8,2817.5,2758.0,2847.4
528,CLV05_MEUR,NSA,P31_S15,IS,48.2,48.6,49.0,49.4,47.2,47.6,...,83.4,94.4,94.6,93.2,84.2,96.4,96.7,94.9,86.1,96.6
568,CLV05_MEUR,NSA,P32_S13,IS,243.5,249.2,257.8,263.6,246.5,252.3,...,435.0,444.4,438.8,452.8,439.0,453.8,448.3,462.8,449.2,455.4
606,CLV05_MEUR,NSA,P3_P5,IS,NaN,NaN,NaN,NaN,2117.6,2284.4,...,4952.8,5009.9,4980.1,5061.0,4915.4,5158.2,4998.3,5290.4,5227.6,5360.5
644,CLV05_MEUR,NSA,P3_P6,IS,NaN,NaN,NaN,NaN,2765.2,2973.3,...,6879.6,7263.8,7513.5,7357.4,6815.3,7349.7,7458.6,7542.8,7218.3,7550.9
683,CLV05_MEUR,NSA,P3_S13,IS,628.0,642.7,664.9,679.8,635.8,650.8,...,1121.8,1146.1,1131.7,1167.8,1132.3,1170.4,1156.2,1193.6,1158.4,1174.4


,unit,coicop,geo,1995-01-01T00:00:00,1996-01-01T00:00:00,1997-01-01T00:00:00,1998-01-01T00:00:00,1999-01-01T00:00:00,2000-01-01T00:00:00,2001-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
23,CLV05_MEUR,CP01,IS,637.0,663.7,697.2,731.0,745.8,748.1,770.7,...,1108.6,1163.7,1238.0,1311.1,1319.1,1338.8,1392.3,1491.5,1491.1,1547.7
61,CLV05_MEUR,CP011,IS,572.6,598.3,628.3,654.5,669.6,665.6,683.0,...,967.7,1014.1,1074.9,1121.5,1124.2,1131.1,1167.8,1246.6,1234.2,1290.0
99,CLV05_MEUR,CP012,IS,65.0,65.8,69.4,76.8,76.6,82.6,87.8,...,141.3,150.2,164.3,193.3,199.1,213.2,231.2,252.9,266.7,266.4
138,CLV05_MEUR,CP02,IS,268.3,277.4,284.0,296.8,316.8,323.1,328.4,...,326.2,356.1,359.5,364.6,375.2,423.5,423.5,402.4,388.2,382.1
176,CLV05_MEUR,CP021,IS,107.1,112.1,118.5,130.0,141.6,149.1,157.8,...,195.0,229.2,242.2,246.8,256.0,301.6,302.5,281.2,275.6,265.0
213,CLV05_MEUR,CP022,IS,152.4,154.3,150.4,145.4,150.9,148.6,142.4,...,91.9,88.1,79.3,78.5,79.7,85.9,83.0,78.8,71.4,74.0
247,CLV05_MEUR,CP023,IS,22.9,24.2,25.7,28.4,30.8,30.7,31.2,...,41.5,43.8,45.8,48.1,49.0,48.0,51.7,55.9,56.2,56.2
285,CLV05_MEUR,CP03,IS,198.9,199.4,210.2,246.3,263.1,276.6,282.0,...,363.7,409.4,450.6,510.2,527.5,472.9,587.8,565.4,576.1,583.9
323,CLV05_MEUR,CP031,IS,170.1,170.5,179.7,210.5,226.9,239.5,244.8,...,300.6,343.4,385.4,448.6,465.9,404.2,503.0,465.2,478.8,486.0
361,CLV05_MEUR,CP032,IS,28.9,29.0,30.6,35.9,36.6,37.6,37.8,...,61.2,65.0,65.8,65.6,66.3,69.5,85.9,97.7,95.1,95.9


,na_item,unit,geo,2013-01-01T00:00:00,2014-01-01T00:00:00,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
20,B1GQ,CP_EUR_HAB,IS,38460.0,42350.0,49180.0,57900.0,65850.0,65330.0,63430.0,54190.0,61360.0,74950.0,77210.0,79480.0
61,B1GQ,CP_MEUR,IS,12222.7,13611.7,15960.5,19067.0,22211.8,22591.2,22315.1,19302.5,22188.1,27738.9,29301.6,30726.8


,unit,coicop,geo,1996-01-01T00:00:00,1996-02-01T00:00:00,1996-03-01T00:00:00,1996-04-01T00:00:00,1996-05-01T00:00:00,1996-06-01T00:00:00,1996-07-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
23,I05,CP00,IS,76.08,76.23,76.39,76.70,77.00,77.00,77.08,...,228.12,228.06,230.47,231.53,233.24,234.33,238.61,239.75,239.35,238.41
64,I05,CP01,IS,80.85,81.10,81.84,82.42,82.75,82.25,82.09,...,264.05,266.09,269.17,271.19,273.26,275.05,276.62,276.82,277.10,277.89
105,I05,CP011,IS,79.73,80.06,80.87,81.53,81.94,81.45,81.28,...,270.42,271.51,274.43,276.67,278.84,280.79,281.84,281.87,282.15,283.23
145,I05,CP0111,IS,78.51,78.91,79.47,79.23,79.31,79.87,80.59,...,281.60,282.35,285.87,286.34,287.33,289.47,289.55,290.08,289.03,289.12
217,I05,CP0112,IS,78.18,77.47,78.49,77.54,79.04,79.20,78.49,...,239.32,240.34,245.05,247.89,251.60,252.18,253.94,253.08,252.96,253.94
285,I05,CP0113,IS,60.84,60.17,59.87,59.39,60.60,60.84,61.32,...,285.68,288.14,288.30,294.23,295.57,296.69,297.85,298.64,300.84,297.97
343,I05,CP0114,IS,81.03,81.70,84.21,83.71,84.46,83.71,84.21,...,316.96,316.94,317.27,320.48,320.89,328.75,331.03,331.05,331.47,334.68
408,I05,CP0115,IS,94.04,93.57,93.57,93.48,94.23,93.48,94.23,...,407.16,409.75,411.53,415.56,416.08,420.09,422.18,422.51,421.21,422.83
465,I05,CP0116,IS,97.33,97.83,98.75,99.77,101.30,103.54,103.33,...,303.30,300.64,294.60,293.78,298.44,301.31,303.85,299.34,299.03,301.31
518,I05,CP0117,IS,99.23,102.88,101.63,115.73,110.30,100.38,97.24,...,267.05,271.16,278.25,277.76,280.52,274.91,274.58,273.45,274.44,276.19


,unit,coicop,geo,1997-01-01T00:00:00,1997-02-01T00:00:00,1997-03-01T00:00:00,1997-04-01T00:00:00,1997-05-01T00:00:00,1997-06-01T00:00:00,1997-07-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
227,RCH_A,CP00,IS,2.0,1.9,1.7,2.2,1.5,1.6,1.6,...,3.6,3.7,3.4,3.4,3.9,3.5,3.9,3.7,3.7,4.3
272,RCH_A,CP01,IS,2.6,2.4,1.0,4.2,2.5,2.5,2.8,...,4.0,4.2,4.7,5.1,5.7,6.0,6.1,5.2,5.6,6.2
317,RCH_A,CP011,IS,3.1,2.7,1.1,4.4,2.7,2.4,2.5,...,4.3,4.2,4.7,5.2,5.8,6.0,5.9,4.8,5.4,6.0
361,RCH_A,CP0111,IS,2.4,2.5,2.1,5.6,6.4,5.3,4.8,...,2.6,2.5,3.2,3.2,3.2,3.4,2.7,2.4,2.2,3.1
405,RCH_A,CP01111,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.3,8.8,8.6,7.3,6.3,4.4,3.2,0.8,0.4,1.1
448,RCH_A,CP01112,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.2,-6.2,-4.1,-4.1,-4.7,-3.9,-3.7,-4.3,-3.0,0.3
491,RCH_A,CP01113,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.7,4.4,5.1,5.0,3.5,4.3,2.6,2.9,2.8,3.9
534,RCH_A,CP01114,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.9,3.2,3.7,3.6,4.4,4.5,4.1,3.8,3.5,4.3
577,RCH_A,CP01115,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.4,-3.4,-1.7,-1.0,-0.5,-0.8,-1.1,-1.8,-1.7,0.0
619,RCH_A,CP01116,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.4,-3.7,-2.8,-2.8,-5.0,-3.2,-3.3,-4.1,-3.8,-1.4


...

,unit,sex,age,citizen,geo,1998-04-01T00:00:00,1999-04-01T00:00:00,2000-04-01T00:00:00,2001-04-01T00:00:00,2002-04-01T00:00:00,...,2023-01-01T00:00:00,2023-04-01T00:00:00,2023-07-01T00:00:00,2023-10-01T00:00:00,2024-01-01T00:00:00,2024-04-01T00:00:00,2024-07-01T00:00:00,2024-10-01T00:00:00,2025-01-01T00:00:00,2025-04-01T00:00:00
19,PC,F,Y15-19,EU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,PC,F,Y15-19,FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,PC,F,Y15-19,NAT,IS,61.8,66.7,76.6,73.0,61.7,...,67.9,73.3,80.5,66.4,73.3,68.3,75.0,60.3,64.0,68.3
130,PC,F,Y15-19,NEU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,PC,F,Y15-19,NRP,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,PC,F,Y15-19,TOTAL,IS,61.4,67.3,76.9,73.4,62.5,...,67.9,73.3,76.7,66.4,72.8,69.1,73.2,60.2,64.0,68.6
262,PC,F,Y15-24,EU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,76.6,NaN,NaN,NaN,67.0,78.8,NaN,NaN,NaN,NaN
299,PC,F,Y15-24,FOR,IS,NaN,NaN,NaN,NaN,NaN,...,76.6,70.4,69.8,NaN,69.8,82.2,NaN,91.0,NaN,NaN
336,PC,F,Y15-24,NAT,IS,68.4,71.0,77.5,75.0,66.9,...,71.2,75.2,82.8,73.9,77.3,77.9,78.4,68.0,74.1,82.7
373,PC,F,Y15-24,NEU27_2020_FOR,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,s_adj,age,unit,sex,geo,2003-01-01T00:00:00,2003-02-01T00:00:00,2003-03-01T00:00:00,2003-04-01T00:00:00,2003-05-01T00:00:00,...,2024-12-01T00:00:00,2025-01-01T00:00:00,2025-02-01T00:00:00,2025-03-01T00:00:00,2025-04-01T00:00:00,2025-05-01T00:00:00,2025-06-01T00:00:00,2025-07-01T00:00:00,2025-08-01T00:00:00,2025-09-01T00:00:00
19,NSA,TOTAL,PC_ACT,F,IS,4.5,3.1,2.6,2.9,6.3,...,3.9,3.0,7.2,4.2,3.6,4.5,2.4,2.9,4.0,3.1
56,NSA,TOTAL,PC_ACT,M,IS,6.0,5.2,3.5,2.8,6.4,...,3.5,7.1,5.0,3.2,4.3,4.5,2.0,3.5,6.4,3.2
93,NSA,TOTAL,PC_ACT,T,IS,5.3,4.2,3.1,2.9,6.4,...,3.7,5.2,6.0,3.7,4.0,4.5,2.2,3.2,5.3,3.2
130,NSA,TOTAL,THS_PER,F,IS,3.0,2.0,2.0,2.0,5.0,...,4.0,3.0,8.0,5.0,4.0,5.0,3.0,3.0,4.0,3.0
167,NSA,TOTAL,THS_PER,M,IS,5.0,4.0,3.0,2.0,6.0,...,4.0,9.0,6.0,4.0,5.0,6.0,3.0,5.0,8.0,4.0
204,NSA,TOTAL,THS_PER,T,IS,8.0,6.0,5.0,4.0,10.0,...,9.0,12.0,14.0,9.0,9.0,11.0,5.0,8.0,13.0,7.0
241,NSA,Y25-74,PC_ACT,F,IS,3.4,2.8,2.3,1.2,3.6,...,1.4,2.6,4.8,2.6,2.0,3.0,2.0,3.2,3.0,2.3
278,NSA,Y25-74,PC_ACT,M,IS,4.7,2.7,3.3,2.0,3.1,...,2.5,5.8,4.1,1.6,3.2,3.8,2.1,3.7,5.5,2.0
315,NSA,Y25-74,PC_ACT,T,IS,4.1,2.7,2.8,1.6,3.3,...,2.0,4.3,4.4,2.0,2.6,3.4,2.0,3.5,4.3,2.1
352,NSA,Y25-74,THS_PER,F,IS,2.0,2.0,1.0,1.0,2.0,...,1.0,2.0,4.0,2.0,2.0,3.0,2.0,3.0,3.0,2.0


,sex,age,unit,isced11,geo,1999-01-01T00:00:00,2000-01-01T00:00:00,2001-01-01T00:00:00,2002-01-01T00:00:00,2003-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
19,F,Y15-64,PC,ED0-2,IS,56.9,54.7,55.1,53.4,49.5,...,31.3,28.1,28.8,26.7,24.9,25.3,22.1,22.2,20.8,20.2
57,F,Y15-64,PC,ED3-8,IS,43.1,45.3,44.9,46.6,50.5,...,68.7,71.9,71.2,73.3,75.1,74.7,77.9,77.8,79.2,79.8
95,F,Y15-64,PC,ED3_4,IS,25.2,25.3,25.4,25.6,24.8,...,30.8,31.8,29.0,29.1,30.6,31.8,35.1,32.3,32.5,29.5
133,F,Y15-64,PC,ED3_4GEN,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21.2,18.4,19.0,16.7
168,F,Y15-64,PC,ED3_4VOC,IS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.9,12.5,12.2,12.4
203,F,Y15-64,PC,ED5-8,IS,18.0,20.0,19.5,21.0,25.8,...,37.9,40.1,42.2,44.2,44.5,42.9,42.8,45.5,46.6,50.2
241,F,Y20-24,PC,ED0-2,IS,59.0,52.5,46.7,43.1,43.7,...,30.0,26.7,28.3,29.2,24.1,24.5,25.9,27.3,17.9,22.0
279,F,Y20-24,PC,ED3-8,IS,41.0,47.5,53.3,56.9,56.3,...,70.0,73.3,71.7,70.8,75.9,75.5,74.1,72.7,82.1,78.0
317,F,Y20-24,PC,ED3_4,IS,38.2,45.3,49.5,53.8,52.8,...,66.6,66.1,62.6,63.9,71.2,66.5,66.8,62.7,68.3,61.6
355,F,Y20-24,PC,ED3_4GEN,IS,NaN,NaN,NaN,NaN,NaN,...,63.2,32.5,7.9,42.1,65.2,45.3,62.8,59.2,62.6,53.1


,unit,estruct,ecase,geo,2000-01-01T00:00:00,2001-01-01T00:00:00,2002-01-01T00:00:00,2003-01-01T00:00:00,2004-01-01T00:00:00,2005-01-01T00:00:00,...,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00,2024-01-01T00:00:00
21,EUR,FAM,CPL_CH2_AW100,IS,2748.12,2546.94,2160.04,2095.47,2038.85,2614.92,...,2057.14,2378.68,3219.15,4201.64,4116.35,4311.82,4761.96,5032.06,4029.24,5082.80
60,EUR,FAM,CPL_CH2_AW100_100,IS,6.32,-21.73,-291.53,-510.32,-737.37,-941.36,...,0.00,0.00,0.00,0.00,0.00,388.12,0.00,0.00,0.00,1265.24
98,EUR,FAM,CPL_CH2_AW100_33,IS,1843.33,1699.28,1351.02,1235.56,1122.70,1441.34,...,945.71,1045.64,1553.49,2161.54,2041.55,2301.18,2438.77,NaN,NaN,NaN
137,EUR,FAM,CPL_CH2_AW100_67,IS,911.11,825.93,517.49,349.59,178.78,232.21,...,0.00,0.00,175.10,493.93,428.09,801.71,992.84,1376.23,541.31,2267.65
176,EUR,FAM,CPL_NCH_AW100_100,IS,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
214,EUR,FAM,CPL_NCH_AW100_33,IS,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
253,EUR,FAM,P1_CH2_AW67,IS,4706.10,4281.97,4048.59,4055.91,4082.18,5069.76,...,3695.05,4148.99,5238.17,5704.83,5352.11,5274.08,5744.70,6056.66,5172.20,6297.31
292,EUR,FAM,P1_NCH_AW100,IS,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
331,EUR,FAM,P1_NCH_AW125,IS,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
370,EUR,FAM,P1_NCH_AW167,IS,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


,unit,nace_r2,geo,2007-01-01T00:00:00,2008-01-01T00:00:00,2009-01-01T00:00:00,2010-01-01T00:00:00,2011-01-01T00:00:00,2012-01-01T00:00:00,2013-01-01T00:00:00,2014-01-01T00:00:00,2015-01-01T00:00:00,2016-01-01T00:00:00,2017-01-01T00:00:00,2018-01-01T00:00:00,2019-01-01T00:00:00,2020-01-01T00:00:00,2021-01-01T00:00:00,2022-01-01T00:00:00,2023-01-01T00:00:00
52,PC,B-N,IS,24.3,21.5,18.9,16.5,16.4,15.7,16.5,15.9,16.4,15.7,14.7,15.8,14.6,11.4,13.3,13.0,13.8
85,PC,B-S,IS,23.4,20.5,18.0,17.5,17.5,17.2,18.5,16.4,17.0,15.6,15.0,13.6,13.8,11.7,9.9,8.7,9.3
123,PC,B-S_X_O,IS,24.0,20.7,18.0,17.7,17.8,17.7,19.0,16.7,17.5,15.8,15.3,13.8,14.1,12.0,10.2,9.0,9.8
159,PC,C,IS,26.1,22.6,21.4,23.3,20.7,19.8,19.1,15.3,16.7,15.1,17.0,16.7,12.8,13.4,12.3,11.4,12.0
195,PC,D,IS,NaN,18.3,19.1,14.0,11.2,14.8,12.6,11.7,12.0,13.3,9.1,8.8,9.6,6.5,3.4,3.9,3.1
231,PC,E,IS,NaN,-5.4,1.6,-2.7,0.8,-2.8,4.4,5.9,-6.6,-0.9,-7.4,-9.2,-11.7,-11.4,-6.6,-9.4,-1.1
267,PC,F,IS,-6.6,-11.9,-27.3,-18.6,-9.1,-12.9,-2.3,0.2,1.2,-10.9,-6.5,-7.4,-12.6,-5.5,-9.4,-10.1,-17.9
303,PC,G,IS,25.8,22.3,23.4,20.6,19.2,18.4,18.1,18.3,19.3,18.0,16.5,14.0,13.8,9.2,9.1,6.8,7.4
339,PC,H,IS,24.7,23.0,20.0,16.1,16.5,15.4,18.4,18.6,18.5,18.6,14.8,14.3,16.2,11.2,13.8,15.4,19.7
375,PC,I,IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.9,7.3,7.0,8.4,6.4,4.9


In [5]:
for frame in DATAFRAMES:
    column_names = frame.dataframe.columns.tolist()
    time_period_index = column_names.index("geo")
    data_columns = column_names[time_period_index + 1 :]
    id_cols = column_names[: time_period_index + 1]

    df_melted = frame.dataframe.melt(id_vars=id_cols, value_vars=data_columns, var_name="TIME_PERIOD", value_name="value")
    df_melted = df_melted.drop(columns=["geo"])
    frame.dataframe = df_melted
    display(frame.dataframe.head(10))

    os.makedirs("melted_data", exist_ok=True)
    frame.dataframe.to_csv(os.path.join("melted_data", frame.file_name), index=False)


,unit,s_adj,na_item,TIME_PERIOD,value
0,CLV05_MEUR,NSA,B1GQ,1995-01-01T00:00:00,2030.6
1,CLV05_MEUR,NSA,P3,1995-01-01T00:00:00,1656.3
2,CLV05_MEUR,NSA,P31_S13,1995-01-01T00:00:00,384.5
3,CLV05_MEUR,NSA,P31_S14,1995-01-01T00:00:00,1004.4
4,CLV05_MEUR,NSA,P31_S14_S15,1995-01-01T00:00:00,1051.2
5,CLV05_MEUR,NSA,P31_S15,1995-01-01T00:00:00,48.2
6,CLV05_MEUR,NSA,P32_S13,1995-01-01T00:00:00,243.5
7,CLV05_MEUR,NSA,P3_P5,1995-01-01T00:00:00,NaN
8,CLV05_MEUR,NSA,P3_P6,1995-01-01T00:00:00,NaN
9,CLV05_MEUR,NSA,P3_S13,1995-01-01T00:00:00,628.0


,unit,coicop,TIME_PERIOD,value
0,CLV05_MEUR,CP01,1995-01-01T00:00:00,637.0
1,CLV05_MEUR,CP011,1995-01-01T00:00:00,572.6
2,CLV05_MEUR,CP012,1995-01-01T00:00:00,65.0
3,CLV05_MEUR,CP02,1995-01-01T00:00:00,268.3
4,CLV05_MEUR,CP021,1995-01-01T00:00:00,107.1
5,CLV05_MEUR,CP022,1995-01-01T00:00:00,152.4
6,CLV05_MEUR,CP023,1995-01-01T00:00:00,22.9
7,CLV05_MEUR,CP03,1995-01-01T00:00:00,198.9
8,CLV05_MEUR,CP031,1995-01-01T00:00:00,170.1
9,CLV05_MEUR,CP032,1995-01-01T00:00:00,28.9


,na_item,unit,TIME_PERIOD,value
0,B1GQ,CP_EUR_HAB,2013-01-01T00:00:00,38460.0
1,B1GQ,CP_MEUR,2013-01-01T00:00:00,12222.7
2,B1GQ,CP_EUR_HAB,2014-01-01T00:00:00,42350.0
3,B1GQ,CP_MEUR,2014-01-01T00:00:00,13611.7
4,B1GQ,CP_EUR_HAB,2015-01-01T00:00:00,49180.0
5,B1GQ,CP_MEUR,2015-01-01T00:00:00,15960.5
6,B1GQ,CP_EUR_HAB,2016-01-01T00:00:00,57900.0
7,B1GQ,CP_MEUR,2016-01-01T00:00:00,19067.0
8,B1GQ,CP_EUR_HAB,2017-01-01T00:00:00,65850.0
9,B1GQ,CP_MEUR,2017-01-01T00:00:00,22211.8


,unit,coicop,TIME_PERIOD,value
0,I05,CP00,1996-01-01T00:00:00,76.08
1,I05,CP01,1996-01-01T00:00:00,80.85
2,I05,CP011,1996-01-01T00:00:00,79.73
3,I05,CP0111,1996-01-01T00:00:00,78.51
4,I05,CP0112,1996-01-01T00:00:00,78.18
5,I05,CP0113,1996-01-01T00:00:00,60.84
6,I05,CP0114,1996-01-01T00:00:00,81.03
7,I05,CP0115,1996-01-01T00:00:00,94.04
8,I05,CP0116,1996-01-01T00:00:00,97.33
9,I05,CP0117,1996-01-01T00:00:00,99.23


,unit,coicop,TIME_PERIOD,value
0,RCH_A,CP00,1997-01-01T00:00:00,2.0
1,RCH_A,CP01,1997-01-01T00:00:00,2.6
2,RCH_A,CP011,1997-01-01T00:00:00,3.1
3,RCH_A,CP0111,1997-01-01T00:00:00,2.4
4,RCH_A,CP01111,1997-01-01T00:00:00,NaN
5,RCH_A,CP01112,1997-01-01T00:00:00,NaN
6,RCH_A,CP01113,1997-01-01T00:00:00,NaN
7,RCH_A,CP01114,1997-01-01T00:00:00,NaN
8,RCH_A,CP01115,1997-01-01T00:00:00,NaN
9,RCH_A,CP01116,1997-01-01T00:00:00,NaN


...

,unit,sex,age,citizen,TIME_PERIOD,value
0,PC,F,Y15-19,EU27_2020_FOR,1998-04-01T00:00:00,NaN
1,PC,F,Y15-19,FOR,1998-04-01T00:00:00,NaN
2,PC,F,Y15-19,NAT,1998-04-01T00:00:00,61.8
3,PC,F,Y15-19,NEU27_2020_FOR,1998-04-01T00:00:00,NaN
4,PC,F,Y15-19,NRP,1998-04-01T00:00:00,NaN
5,PC,F,Y15-19,TOTAL,1998-04-01T00:00:00,61.4
6,PC,F,Y15-24,EU27_2020_FOR,1998-04-01T00:00:00,NaN
7,PC,F,Y15-24,FOR,1998-04-01T00:00:00,NaN
8,PC,F,Y15-24,NAT,1998-04-01T00:00:00,68.4
9,PC,F,Y15-24,NEU27_2020_FOR,1998-04-01T00:00:00,NaN


,s_adj,age,unit,sex,TIME_PERIOD,value
0,NSA,TOTAL,PC_ACT,F,2003-01-01T00:00:00,4.5
1,NSA,TOTAL,PC_ACT,M,2003-01-01T00:00:00,6.0
2,NSA,TOTAL,PC_ACT,T,2003-01-01T00:00:00,5.3
3,NSA,TOTAL,THS_PER,F,2003-01-01T00:00:00,3.0
4,NSA,TOTAL,THS_PER,M,2003-01-01T00:00:00,5.0
5,NSA,TOTAL,THS_PER,T,2003-01-01T00:00:00,8.0
6,NSA,Y25-74,PC_ACT,F,2003-01-01T00:00:00,3.4
7,NSA,Y25-74,PC_ACT,M,2003-01-01T00:00:00,4.7
8,NSA,Y25-74,PC_ACT,T,2003-01-01T00:00:00,4.1
9,NSA,Y25-74,THS_PER,F,2003-01-01T00:00:00,2.0


,sex,age,unit,isced11,TIME_PERIOD,value
0,F,Y15-64,PC,ED0-2,1999-01-01T00:00:00,56.9
1,F,Y15-64,PC,ED3-8,1999-01-01T00:00:00,43.1
2,F,Y15-64,PC,ED3_4,1999-01-01T00:00:00,25.2
3,F,Y15-64,PC,ED3_4GEN,1999-01-01T00:00:00,NaN
4,F,Y15-64,PC,ED3_4VOC,1999-01-01T00:00:00,NaN
5,F,Y15-64,PC,ED5-8,1999-01-01T00:00:00,18.0
6,F,Y20-24,PC,ED0-2,1999-01-01T00:00:00,59.0
7,F,Y20-24,PC,ED3-8,1999-01-01T00:00:00,41.0
8,F,Y20-24,PC,ED3_4,1999-01-01T00:00:00,38.2
9,F,Y20-24,PC,ED3_4GEN,1999-01-01T00:00:00,NaN


,unit,estruct,ecase,TIME_PERIOD,value
0,EUR,FAM,CPL_CH2_AW100,2000-01-01T00:00:00,2748.12
1,EUR,FAM,CPL_CH2_AW100_100,2000-01-01T00:00:00,6.32
2,EUR,FAM,CPL_CH2_AW100_33,2000-01-01T00:00:00,1843.33
3,EUR,FAM,CPL_CH2_AW100_67,2000-01-01T00:00:00,911.11
4,EUR,FAM,CPL_NCH_AW100_100,2000-01-01T00:00:00,0.00
5,EUR,FAM,CPL_NCH_AW100_33,2000-01-01T00:00:00,0.00
6,EUR,FAM,P1_CH2_AW67,2000-01-01T00:00:00,4706.10
7,EUR,FAM,P1_NCH_AW100,2000-01-01T00:00:00,0.00
8,EUR,FAM,P1_NCH_AW125,2000-01-01T00:00:00,0.00
9,EUR,FAM,P1_NCH_AW167,2000-01-01T00:00:00,0.00


,unit,nace_r2,TIME_PERIOD,value
0,PC,B-N,2007-01-01T00:00:00,24.3
1,PC,B-S,2007-01-01T00:00:00,23.4
2,PC,B-S_X_O,2007-01-01T00:00:00,24.0
3,PC,C,2007-01-01T00:00:00,26.1
4,PC,D,2007-01-01T00:00:00,NaN
5,PC,E,2007-01-01T00:00:00,NaN
6,PC,F,2007-01-01T00:00:00,-6.6
7,PC,G,2007-01-01T00:00:00,25.8
8,PC,H,2007-01-01T00:00:00,24.7
9,PC,I,2007-01-01T00:00:00,NaN


In [11]:
for frame in DATAFRAMES:
    df = frame.dataframe.drop(columns=["TIME_PERIOD", "value"]).drop_duplicates()

    # if there is 'coicop' column, remove the values starting with CP
    if "coicop" in df.columns:
        df = df[~df["coicop"].astype(str).str.startswith("CP")]

    unique_values = {col: df[col].unique().tolist() for col in df.columns}
    print(f"Unique values for {frame.name}:")
    for col, values in unique_values.items():
        print(f"  {col}: {values}")
    print("\n")

Unique values for Quarterly GDP:
  unit: ['CLV05_MEUR', 'CLV05_MNAC', 'CLV10_MEUR', 'CLV10_MNAC', 'CLV15_MEUR', 'CLV15_MNAC', 'CLV20_MEUR', 'CLV20_MNAC', 'CLV_I05', 'CLV_I10', 'CLV_I15', 'CLV_I20', 'CLV_PCH_ANN', 'CLV_PCH_PRE', 'CLV_PCH_SM', 'CON_PPCH_PRE', 'CON_PPCH_SM', 'CP_MEUR', 'CP_MNAC', 'PC_GDP', 'PD05_EUR', 'PD05_NAC', 'PD10_EUR', 'PD10_NAC', 'PD15_EUR', 'PD15_NAC', 'PD20_EUR', 'PD20_NAC', 'PD_PCH_PRE_EUR', 'PD_PCH_PRE_NAC', 'PD_PCH_SM_EUR', 'PD_PCH_SM_NAC', 'PYP_MEUR', 'PYP_MNAC']
  s_adj: ['NSA', 'SCA']
  na_item: ['B1GQ', 'P3', 'P31_S13', 'P31_S14', 'P31_S14_S15', 'P31_S15', 'P32_S13', 'P3_P5', 'P3_P6', 'P3_S13', 'P41', 'P51G', 'P5G', 'P6', 'P61', 'P62', 'P7', 'P71', 'P72', 'P52_P53', 'P61X71', 'P62X72', 'P6X7', 'P52', 'YA0']


Unique values for Annual Household Disposable Income:
  unit: ['CLV05_MEUR', 'CLV05_MNAC', 'CLV10_MEUR', 'CLV10_MNAC', 'CLV15_MEUR', 'CLV15_MNAC', 'CLV20_MEUR', 'CLV20_MNAC', 'CLV_I10', 'CLV_I15', 'CLV_I20', 'CLV_PCH_PRE', 'CP_EUR_HAB', 'CP_MEUR', 'CP

In [13]:
for frame in DATAFRAMES:
    # get all the unique column combinations except TIME_PERIOD and value
    unique_combinations = frame.dataframe.drop(columns=["TIME_PERIOD", "value"]).drop_duplicates()

    if "coicop" in unique_combinations.columns:
        unique_combinations = unique_combinations[~unique_combinations["coicop"].astype(str).str.startswith("CP")]

    # get the number of values for each combination
    unique_combinations["value_count"] = unique_combinations.apply(
        lambda row: frame.dataframe[
            (frame.dataframe.drop(columns=["TIME_PERIOD", "value"]) == row).all(axis=1)
        ].shape[0], axis=1
    )

    print(f"Unique column combinations for {frame.name}:")
    display(unique_combinations)

Unique column combinations for Quarterly GDP:


,unit,s_adj,na_item,value_count
0,CLV05_MEUR,NSA,B1GQ,122
1,CLV05_MEUR,NSA,P3,122
2,CLV05_MEUR,NSA,P31_S13,122
3,CLV05_MEUR,NSA,P31_S14,122
4,CLV05_MEUR,NSA,P31_S14_S15,122
...,...,...,...,...
1190,PYP_MNAC,NSA,P6X7,122
1191,PYP_MNAC,NSA,P7,122
1192,PYP_MNAC,NSA,P71,122
1193,PYP_MNAC,NSA,P72,122


Unique column combinations for Annual Household Disposable Income:


,unit,coicop,value_count
60,CLV05_MEUR,TOTAL,30
121,CLV05_MNAC,TOTAL,30
182,CLV10_MEUR,TOTAL,30
243,CLV10_MNAC,TOTAL,30
304,CLV15_MEUR,TOTAL,30
365,CLV15_MNAC,TOTAL,30
426,CLV20_MEUR,TOTAL,30
487,CLV20_MNAC,TOTAL,30
548,CLV_I10,TOTAL,30
609,CLV_I15,TOTAL,30


Unique column combinations for Annual GDP at Market Prices:


...

,sex,age,unit,isced11,value_count
0,F,Y15-64,PC,ED0-2,26
1,F,Y15-64,PC,ED3-8,26
2,F,Y15-64,PC,ED3_4,26
3,F,Y15-64,PC,ED3_4GEN,26
4,F,Y15-64,PC,ED3_4VOC,26
...,...,...,...,...,...
175,T,Y55-64,PC,ED3-8,26
176,T,Y55-64,PC,ED3_4,26
177,T,Y55-64,PC,ED3_4GEN,26
178,T,Y55-64,PC,ED3_4VOC,26


Unique column combinations for Annual net earnings:


,unit,estruct,ecase,value_count
0,EUR,FAM,CPL_CH2_AW100,25
1,EUR,FAM,CPL_CH2_AW100_100,25
2,EUR,FAM,CPL_CH2_AW100_33,25
3,EUR,FAM,CPL_CH2_AW100_67,25
4,EUR,FAM,CPL_NCH_AW100_100,25
...,...,...,...,...
229,PPS,TOTAL,P1_NCH_AW125,25
230,PPS,TOTAL,P1_NCH_AW167,25
231,PPS,TOTAL,P1_NCH_AW50,25
232,PPS,TOTAL,P1_NCH_AW67,25


Unique column combinations for Annual gender pay gap in unadjusted form:


,unit,nace_r2,value_count
0,PC,B-N,17
1,PC,B-S,17
2,PC,B-S_X_O,17
3,PC,C,17
4,PC,D,17
5,PC,E,17
6,PC,F,17
7,PC,G,17
8,PC,H,17
9,PC,I,17


In [ ]:
for frame in DATAFRAMES:
    quantiles_25 = []
    quantiles_75 = []
    interquartile_ranges = []

    for unit in df["unit"].unique():
        unit_data = df[df["unit"] == unit]
        quantiles_25.append(unit_data["value"].quantile(0.25))
        quantiles_75.append(unit_data["value"].quantile(0.75))
        interquartile_ranges.append(quantiles_75[-1] - quantiles_25[-1])

    # quantile_25 = frame.dataframe["value"].quantile(0.25)
    # quantile_75 = frame.dataframe["value"].quantile(0.75)
    # interquartile_range = quantile_75 - quantile_25
    # outliers = frame.dataframe[(frame.dataframe["value"] < (quantile_25 - 1.5 * interquartile_range)) | (frame.dataframe["value"] > (quantile_75 + 1.5 * interquartile_range))]

    # display(outliers.head(10))

    # plt.figure(figsize=(10, 6))
    # sns.boxplot(x=frame.dataframe["value"])
    # plt.title(f"{frame.name} - Outlier Detection")
    # plt.show()


_IncompleteInputError: incomplete input (1399184074.py, line 13)